# Model snow depth using terrain parameters

In [2]:
import os
import sys
import xdem
import geoutils as gu
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance
import joblib
import seaborn as sns
import json

## Define input and output files

In [20]:
code_dir = '/Users/raineyaberle/Research/PhD/SnowDEMs/skysat_stereo_snow/scripts'
data_dir = '/Volumes/LaCie/raineyaberle/Research/PhD/SkySat-Stereo/study-sites'

# Inputs
site_name = "MCS"
date = "20240420"
sd_fn = os.path.join(data_dir, site_name, date, 'post_process', f"ba+DEMuncertainty1mAll_{site_name}_{date}_DEM_GCPshift_snow_depth.tif")
refdem_fn = os.path.join(data_dir, site_name, 'refdem', f"{site_name}_REFDEM_WGS84.tif")
# refdem_fn = os.path.join(data_dir, site_name, 'refdem', 'USGS_LPC_ID_FEMAHQ_2018_D18_merged_filtered.tif')
terrain_dict_fn = os.path.join(data_dir, site_name, 'refdem', 'refdem_terrain_sampling_resolutions.json')

# Outputs
out_dir = os.path.join(data_dir, site_name, date, 'snow_depth_modeling')
training_data_fn = os.path.join(out_dir, 'training_data.csv')
error_fn = os.path.join(out_dir, 'model_error.csv')
model_fn = os.path.join(out_dir, 'trained_model.joblib')

# Check that inputs exist
for fn, name in [[sd_fn, 'Snow depth map'], [refdem_fn, 'Reference DEM'], [terrain_dict_fn, 'Terrain resolutions dictionary']]:
    if not os.path.exists(fn):
        print(f"{name} not found, please correct before continuing.")
# Create output directory
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

# Import processing functions
sys.path.append(code_dir)
import post_process_utils as pprocess

## Construct model training data

In [ ]:
# Check if training data already exist
if not os.path.exists(training_data_fn):
    ### Load input files
    # Snow depth
    sd = xdem.DEM(sd_fn)
    # Reference DEM 
    refdem = xdem.DEM(refdem_fn).reproject(sd)
    # Reference DEM sampling resolutions
    with open(terrain_dict_fn, 'r') as f:
        terrain_dict = json.load(f)
    for key, value in terrain_dict.items():
        terrain_dict[key] = float(value)
    
    ### Calculate terrain parameters
    elev, slope, aspect, tpi, sx = pprocess.calculate_terrain_params(refdem, terrain_dict, sd)
    # Compile into pandas.DataFrame
    training_data = pd.DataFrame({'elevation': refdem.data.ravel(),
                                  'slope': slope.data.ravel(),
                                  'aspect': aspect.data.ravel(),
                                  'topographic_position_index': tpi.data.ravel(),
                                  'Sx': sx.data.ravel(),
                                  'snow_depth': sd.data.ravel()})
    training_data.dropna(inplace=True)
    training_data.reset_index(drop=True, inplace=True)
    
    # Reduce precision (don't need 10 digits of elevation, e.g.)
    training_data = training_data.round(2)
    
    # Save to file
    training_data.to_csv(training_data_fn, index=False)
    print('Training data saved to file:', training_data_fn)
    
    # Plot pairplot
    # fig_fn = os.path.join(out_dir, 'training_data_pairplot.png')
    # fig = sns.pairplot(training_data, corner=True, kind='hist', diag_kind='kde')
    # fig.savefig(fig_fn, dpi=250, bbox_inches='tight')
    # print('Pairplot saved to file:', fig_fn)
    # plt.show()

else:

    training_data = pd.read_csv(training_data_fn)
    print('Training data loaded from file.')

training_data

## Scale input features

In [ ]:
feature_cols = ['elevation', 'slope', 'aspect', 'topographic_position_index', 'Sx']
target_cols = ['snow_depth']
X = training_data[feature_cols]
y = training_data[target_cols].values

# Fit a standard scaler to the feature columns
scaler_fn = os.path.join(out_dir, 'feature_scaler.joblib')
if not os.path.exists(scaler_fn):
    scaler = StandardScaler().fit(X)
    joblib.dump(scaler, scaler_fn)
    print('Feature scaler saved to file:', scaler_fn)
else:
    scaler = joblib.load(scaler_fn)
    print('Feature scaler loaded from file.')
    
# Transform the feature columns
X_scaled = scaler.transform(X)
X_scaled


## Train, tune, and test the Random Forest model

In [ ]:
model_fn = os.path.join(out_dir, 'trained_model.joblib')
if not os.path.exists(model_fn):

    ### Use a subset of the training data
    X_scaled_sub, y_sub = X_scaled[::100,:], y[::100,:]

    best_model = RandomForestRegressor().fit(X_scaled_sub, y_sub)
    joblib.dump(best_model, model_fn)
    print('Trained model saved to file:', model_fn)
    
    # ### Set up the hyperparameter grid
    # # Number of trees
    # n_estimators = [int(x) for x in np.linspace(200, 2000, num = 10)]
    # # Number of features at every split
    # max_features = ['auto', 'sqrt']
    # # Maximum number of levels in tree
    # max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
    # max_depth.append(None)
    # # Minimum number of samples required to split a node
    # min_samples_split = [2, 5, 10]
    # # Minimum number of samples required at each leaf node
    # min_samples_leaf = [1, 2, 4]
    # # Create the random grid
    # param_grid = {'n_estimators': n_estimators,
    #               'max_features': max_features,
    #               'max_depth': max_depth,
    #               'min_samples_split': min_samples_split,
    #               'min_samples_leaf': min_samples_leaf}
    # # print('Hyperparameter Grid:')
    # # print(json.dumps(random_grid, sort_keys=True, indent=4))

    # ### Search for best hyperparameters
    # rf = RandomForestRegressor()
    # rf_random = RandomizedSearchCV(estimator=rf, param_distributions=param_grid, 
    #                             n_iter=100, cv=3, verbose=2, random_state=42, n_jobs=-1)
    # rf_random.fit(X_scaled_sub, y_sub)
    # print('Best parameters:')
    # print(rf_random.best_params_)
    
    # ### Compare to the default (base) model
    # cv_model = rf_random.best_estimator_
    # y_pred = cv_model.predict(X_scaled)
    # cv_rmse = np.sqrt(mean_squared_error(y, y_pred))
    # print(f"Best model RMSE = {np.round(cv_rmse,2)} m")
    # base_model = RandomForestRegressor().fit(X_scaled_sub, y_sub)
    # y_pred = base_model.predict(X_scaled)
    # base_rmse = np.sqrt(mean_squared_error(y, y_pred))
    # print(f"CV model RMSE = {np.round(base_rmse,2)} m")

    # ### Save the better model
    # if base_rmse < cv_rmse:
    #     # Use base model
    #     best_model = base_model
    # else:
    #     # Use CV model
    #     best_model = cv_model
    # joblib.dump(best_model, model_fn)
    # print('Best model saved to file:', model_fn)

else:
    best_model = joblib.load(model_fn)
    print('Best model loaded from file.')
    
best_model


## Estimate feature importances in model

In [ ]:
feature_importances_fn = os.path.join(out_dir, 'feature_importances.csv')
if not os.path.exists(feature_importances_fn):
    # Feature importances via mean decrease in impurity
    mdi_importances = best_model.feature_importances_
    std = np.std([tree.feature_importances_ for tree in best_model.estimators_], axis=0)
    mdi_importances = pd.Series(mdi_importances, index=feature_cols)

    # Feature importances via permutation
    result = permutation_importance(best_model, X_scaled_sub, y_sub, n_repeats=10, random_state=42, n_jobs=2)
    perm_importances = pd.Series(result.importances_mean, index=feature_cols)

    # Save in dataframe
    feature_importances = pd.DataFrame(mdi_importances.values, feature_cols, ['MDI'])
    feature_importances['MDI_std'] = std
    feature_importances['permutation'] = perm_importances
    feature_importances['permutation_std'] = result.importances_std
    feature_importances.to_csv(feature_importances_fn, index=True)
    print('Feature importances saved to file:', feature_importances_fn)
    
    # Plot
    fig, ax = plt.subplots(1, 2, figsize=(10,5))
    mdi_importances.plot.bar(yerr=std, ax=ax[0])
    ax[0].set_title("Feature importances via MDI")
    ax[0].set_ylabel("Mean decrease in impurity")
    perm_importances.plot.bar(yerr=result.importances_std, ax=ax[1])
    ax[1].set_title("Feature importances via permutation")
    ax[1].set_ylabel("Mean accuracy decrease")
    fig.tight_layout()
    plt.show()
    
    # Save figure
    fig_fn = os.path.join(out_dir, 'feature_importances.png')
    fig.savefig(fig_fn, dpi=250, bbox_inches='tight')
    print('Figure saved to file:', fig_fn)
    
else:
    feature_importances = pd.read_csv(feature_importances_fn)
    print('Feature importances loaded from file.')

feature_importances

## Use best model to predict snow depth at site

In [ ]:
# Check if modeled snow depth already exists
sd_pred_fn = os.path.join(out_dir, f'modeled_snow_depth_{site_name}_{date}.tif')
if not os.path.exists(sd_pred_fn):
    # Load SkySat snow depth
    print('Loading input files')
    sd = xdem.DEM(sd_fn)
    refdem = xdem.DEM(refdem_fn).reproject(sd)
    with open(terrain_dict_fn, 'r') as f:
        terrain_dict = json.load(f)
    for key, value in terrain_dict.items():
        terrain_dict[key] = float(value)
    
    # Construct features (terrain parameters)
    elev, slope, aspect, tpi, sx = pprocess.calculate_terrain_params(refdem, terrain_dict, sd)
    
    # Identify real value indices (for reshaping results later)
    print('Constructing features')
    rasters = [elev, slope, aspect, tpi, sx]
    ix = [np.where((np.isfinite(raster.data) & ~np.isnan(raster.data)), True, False) 
        for raster in rasters]
    ireal = np.full(sd.shape, True)
    for ixx in ix:
        ireal = ireal & ixx
    # create df of raster values
    df = pd.DataFrame({'elevation': elev.data[ireal].ravel(),
                       'slope': slope.data[ireal].ravel(),
                       'aspect': aspect.data[ireal].ravel(),
                       'topographic_position_index': tpi.data[ireal].ravel(),
                       'Sx': sx.data[ireal].ravel()})
    df = df.round(2) # round values

    # Scale the input features
    df = scaler.transform(df)

    # Predict snow depth
    print('Modeling snow depth')
    y_pred = best_model.predict(df)

    # Reshape into the raster shape
    snow_depth_pred = np.full(np.shape(ireal), np.nan)
    snow_depth_pred[ireal] = y_pred

    # Reformat as xdem.DEM
    sd_pred = xdem.DEM.from_array(snow_depth_pred, transform=sd.transform, crs=sd.crs, nodata=np.nan)
    
    # Save to file
    sd_pred.save(sd_pred_fn)
    print('Modeled snow depth saved to file:', sd_pred_fn)
    
    # Compare to observations
    fig, ax = plt.subplots(2, 2, figsize=(8,8))
    ax = ax.flatten()
    sd_pred.plot(cmap='Blues', vmin=0, vmax=5, ax=ax[0], add_cbar=False)
    ax[0].set_title('Modeled')
    sd.plot(cmap='Blues', vmin=0, vmax=5, ax=ax[1], cbar_title='Snow depth [m]')
    ax[1].set_title('Observed')
    diff = sd_pred - sd
    diff.plot(cmap='coolwarm_r', vmin=-5, vmax=5, ax=ax[2], cbar_title='Difference [m]')
    ax[2].set_title('Modeled - Observed')
    for axis in ax[0:3]:
        axis.set_xticks([])
        axis.set_yticks([])
    ax[3].hist(diff.data.ravel(), bins=np.arange(-5, 5, step=0.2), facecolor='skyblue', edgecolor='k', linewidth=0.5)
    ax[3].set_xlabel('Modeled - Observed [m]')
    ax[3].set_ylabel('Frequency')
    ax[3].set_yticks([])
    ax[3].axvline(0, color='k', linewidth=1)

    fig.tight_layout()
    plt.show()
    
    # Save figure to file
    fig_fn = os.path.splitext(sd_pred_fn)[0] + '.png'
    fig.savefig(fig_fn, dpi=250, bbox_inches='tight')
    print('Figure saved to file:', fig_fn)

else:
    print('Modeled snow depth already exists, skipping.')


## Use best model to predict snow depth at Mores Creek

In [ ]:
# Check if modeled snow depth already exists
sd_pred_mcs_fn = os.path.join(out_dir, 'modeled_snow_depth_extrapMCS.tif')
if not os.path.exists(sd_pred_mcs_fn):
    # Load training and validation data at MCS
    print('Loading training and validation data')
    sd_lidar_fn = os.path.join(data_dir, 'MCS', 'SNEX_MCS_Lidar', 'SNEX_MCS_Lidar_20240418_SD_V01.0.tif')
    sd_lidar = xdem.DEM(sd_lidar_fn).reproject(res=[1,1])
    if site_name=='MCS':
        mcs_refdem_fn = refdem_fn
    else:
        mcs_refdem_fn = os.path.join(data_dir, 'MCS', 'refdem', 'MCS_REFDEM_WGS84.tif')
    mcs_refdem = xdem.DEM(mcs_refdem_fn).reproject(sd_lidar, nodata=np.nan)
    mcs_terrain_dict_fn = os.path.join(data_dir, 'MCS', 'refdem', 'refdem_terrain_sampling_resolutions.json')
    with open(mcs_terrain_dict_fn, 'r') as f:
        mcs_terrain_dict = json.load(f)
    for key, value in mcs_terrain_dict.items():
        mcs_terrain_dict[key] = float(value)

    # Construct features (terrain parameters)
    elev, slope, aspect, tpi, sx = pprocess.calculate_terrain_params(mcs_refdem, mcs_terrain_dict, sd_lidar)
    
    # Identify real value indices (for reshaping results later)
    print('Constructing features')
    rasters = [elev, slope, aspect, tpi, sx]
    ix = [np.where((np.isfinite(raster.data) & ~np.isnan(raster.data)), True, False) 
        for raster in rasters]
    ireal = np.full(sd_lidar.shape, True)
    for ixx in ix:
        ireal = ireal & ixx

    # create df of raster values
    df = pd.DataFrame({'elevation': elev.data[ireal].ravel(),
                    'slope': slope.data[ireal].ravel(),
                    'aspect': aspect.data[ireal].ravel(),
                    'topographic_position_index': tpi.data[ireal].ravel(),
                    'Sx': sx.data[ireal].ravel()})
    df = df.round(2) # round values

    # Scale the input features
    df = scaler.transform(df)

    # Predict snow depth
    print('Modeling snow depth')
    y_pred = best_model.predict(df)

    # Reshape into the raster shape
    snow_depth_pred = np.full(np.shape(ireal), np.nan)
    snow_depth_pred[ireal] = y_pred

    # Reformat as xdem.DEM
    sd_pred_mcs = xdem.DEM.from_array(snow_depth_pred, transform=sd_lidar.transform, crs=sd_lidar.crs, nodata=np.nan)
    
    # Save to file
    sd_pred_mcs.save(sd_pred_mcs_fn)
    print('Modeled snow depth at MCS saved to file:', sd_pred_mcs_fn)
    
    # Compare to lidar observations
    fig, ax = plt.subplots(2, 2, figsize=(8,8))
    ax = ax.flatten()
    sd_pred_mcs.plot(cmap='Blues', vmin=0, vmax=5, ax=ax[0], add_cbar=False)
    ax[0].set_title('Modeled')
    sd_lidar.plot(cmap='Blues', vmin=0, vmax=5, ax=ax[1], cbar_title='Snow depth [m]')
    ax[1].set_title('Lidar')
    diff = sd_pred_mcs - sd_lidar
    diff.plot(cmap='coolwarm_r', vmin=-5, vmax=5, ax=ax[2], cbar_title='Difference [m]')
    ax[2].set_title('Modeled - Lidar')
    for axis in ax[0:3]:
        axis.set_xticks([])
        axis.set_yticks([])
    ax[3].hist(diff.data.ravel(), bins=np.arange(-5, 5, step=0.2), facecolor='skyblue', edgecolor='k', linewidth=0.5)
    ax[3].set_xlabel('Modeled - Lidar [m]')
    ax[3].set_ylabel('Frequency')
    ax[3].set_yticks([])
    ax[3].axvline(0, color='k', linewidth=1)

    fig.tight_layout()
    plt.show()
    
    # Save figure to file
    fig_fn = os.path.splitext(sd_pred_mcs_fn)[0] + '.png'
    fig.savefig(fig_fn, dpi=250, bbox_inches='tight')
    print('Figure saved to file:', fig_fn)

else:
    print('Modeled snow depth at MCS already exists, skipping.')
